## Simplest ANN Model

一樣是最簡單的 ANN model ，一個 input 層，2 個隱藏層，1個輸出層，輸出層只有 1 個node。

這個 sample 是在 007 的基礎上，將 input 數據轉為百分比，算是 normalize？

但不知道為啥 train 一陣子後會變成 nan.. 和 diff 改成 pct_change 有關...

In [1]:
import torch
import yfinance as yf
import pandas as pd

import time

from IPython.display import clear_output
import matplotlib.pyplot as plt

from utils.rolling_to_list import rolling_to_list

In [2]:
# prepare device
device = torch.device("cpu")
if torch.backends.mps.is_available():
    device = torch.device("mps")

In [3]:
# print up/down in --o--o- format
def puts(arr): print(''.join(["o" if x > 0 else '-' for x in arr]))

In [4]:
tickers = ['^SPX']

In [5]:
# 回看過去的窗口
rolling_back = 120

In [6]:
# 觀察窗口
expect_period = 1 # 5min

In [7]:
model_path = 'model_friday_5.pt'

In [8]:
period = "60d"
interval = "5m"

In [9]:
# # 下載數據，合併成一個 df
# if len(tickers) > 1:
# 	dfs = yf.download(tickers, period=period, interval=interval, group_by='Ticker')
# 	df_global = pd.concat([dfs[k].add_prefix(k + "_") for k in tickers], axis=1)
# else:
# 	df_global = yf.download(tickers, period=period, interval=interval).add_prefix(tickers[0] + "_")

df_global = yf.download(tickers, period=period, interval=interval)
df_global

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-12-04 09:30:00-05:00,4564.370117,4566.580078,4558.390137,4566.580078,4566.580078,127454215
2023-12-04 09:35:00-05:00,4566.959961,4568.049805,4563.319824,4565.990234,4565.990234,88653884
2023-12-04 09:40:00-05:00,4566.009766,4569.209961,4565.479980,4567.479980,4567.479980,82133816
2023-12-04 09:45:00-05:00,4567.410156,4570.919922,4567.410156,4569.609863,4569.609863,87963286
2023-12-04 09:50:00-05:00,4569.680176,4571.839844,4567.049805,4569.140137,4569.140137,78119822
...,...,...,...,...,...,...
2024-02-23 15:35:00-05:00,5090.120117,5093.709961,5089.149902,5092.069824,5092.069824,53058612
2024-02-23 15:40:00-05:00,5092.060059,5092.810059,5091.040039,5091.040039,5091.040039,57465798
2024-02-23 15:45:00-05:00,5091.049805,5095.750000,5090.660156,5095.339844,5095.339844,78725870


In [10]:
def prepare_data(df):
    # 相對於起始位置的改變
    diffs = df.pct_change(rolling_back).add_suffix("_Diff")
    
    # diff to N days ago
    diffs_9 = df.pct_change(9).add_suffix("_Diff9")
    diffs_26 = df.pct_change(26).add_suffix("_Diff26")
    
    hist_diffs = pd.concat([df, diffs, diffs_9, diffs_26], axis=1)
    
    # # 同一個 slot 內相對於 Open 的改變
    # slot_diffs = []
    # for k in tickers:
    #     tg = "Open"
    #     for fg in ["Close", "High", "Low"]:
    #         slot_diff = df["%s_%s" % (k, fg)] - df["%s_%s" % (k, tg)] 
    #         slot_diff.name = "%s_%s_%s" % (k, fg, tg)
    #         slot_diffs.append(slot_diff)

    # slot_diffs = pd.concat(slot_diffs, axis=1)

    slot_diffs = pd.DataFrame()
    slot_diffs['Close_Open'] = (df['Close'] - df['Open']) / df['Open']
    slot_diffs['High_Open'] = (df['High'] - df['Open']) / df['Open']
    slot_diffs['Low_Open'] = (df['Low'] - df['Open']) / df['Open']
    
    all_diffs = pd.concat([hist_diffs, slot_diffs], axis=1).fillna(0)
    # print(all_diffs.columns)
    
    
    # 補充一些技術指標
    extra_series = []
    for k in all_diffs.columns:
        for i in [9, 17]:
            sma_tg = all_diffs[k].rolling(i).mean()
            cross = (all_diffs[k] - sma_tg) / sma_tg
            cross.name = "%s_SMA_c%d" % (k, i)
            extra_series.append(cross)
        
        ema12 = all_diffs[k].ewm(span=12, adjust=False).mean()
        for i in [26]:
            ema_tg = all_diffs[k].ewm(span=i, adjust=False).mean()
            macd = (ema12 - ema_tg) / ema_tg
            macd.name = "%s_MACD_12cross%d" % (k, i)
            extra_series.append(macd)
        
            macd_signal = macd.ewm(span=9, adjust=False).mean()
            macd_signal.name = "%s_signal" % macd.name
            extra_series.append(macd_signal)
    
    extra_series = pd.concat(extra_series, axis=1)
    
    combine = pd.concat([all_diffs, extra_series], axis=1).fillna(0)
    # print(combine)

    # 把過去 N 天的數據拉成一行
    return rolling_to_list(combine, rolling_back)
    
    # # 把過去 N 天的數據拉成一行
    # rolling_back_series = []
    # for k in combine.columns:
    #     # series = pd.Series([w.tolist() for w in combine[k].rolling(rolling_back)], name=k)
    #     sd = pd.DataFrame([w.values for w in combine[k].rolling(rolling_back)])
    #     rolling_back_series.append(sd)
    
    # container = pd.concat(rolling_back_series, axis=1).fillna(0)
    # display(container)
    # return container.values
    
    # 每一行合併成一個 list
    # return container.apply(lambda row: sum(row, []), axis = 1)

In [11]:
loss_function = torch.nn.MSELoss()
# loss_function = torch.nn.L1Loss()
# loss_function = torch.nn.BCELoss()

def prepare_model(input_size):
    # if os.path.isfile(model_path):
    #     model = torch.load(model_path).to(device)
    #     return model
    
    sigmoid_layer = torch.nn.Sigmoid()
    
    layers = []
    layers.append(torch.nn.Linear(input_size, 400))
    layers.append(sigmoid_layer)
    layers.append(torch.nn.Linear(400, 200))
    layers.append(sigmoid_layer)
    layers.append(torch.nn.Linear(200, 1))
    # layers.append(sigmoid_layer)
    
    model = torch.nn.Sequential(*layers).to(device)
    # model = nn_model.MLPClassifier(nn_data.input_size).to(nn_data.device)

    # optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
    optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    torch.save(model, model_path)
    return model, optimizer


In [12]:
def train(epochs, train_set, expect_set, report_period=100):
    for epoch in range(epochs):
        prediction = model(train_set)
        
        loss = loss_function(prediction, expect_set)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if epoch % report_period == 0:
            print(epoch, loss.item())
            
    # save the model
    torch.save(model, model_path)

In [13]:
train_data = prepare_data(df_global)

# there is no actual data for the last set of training data
train_data = train_data[:-expect_period] 
train_data = train_data[rolling_back:]
# expect_set = torch.Tensor(train_data.values).to(device)

# prepare all actual result
actual = df_global['Close'].pct_change(expect_period) * 500
# 往前移動
actual = actual.shift(-expect_period)[:-expect_period]
actual = actual.shift(-rolling_back)[:-rolling_back]

actual = actual.reset_index(drop=True)
actual = actual.values.reshape(-1,1)
# expect_set = torch.Tensor(actual).to(device)

In [14]:
# input_size = 2880
# model, optimizer  = prepare_model(input_size)

# display(pd.DataFrame(train_data[:296]))

# t1 = torch.Tensor(train_data[:296]).to(device)
# t2 = torch.Tensor(actual[:296]).to(device)

# print(t1.shape)
# print(t2.shape)

# # train(1000, t1, t2, 100)
# r1 = model(torch.Tensor(train_data[:296]).to(device))

# loss_function(r1, t2)

In [15]:
# 用前 5 日的數據做相對多次訓練， e.g 5000 次
first_batch_size = 250

In [16]:
df007 = df_global[first_batch_size:]

df007.to_csv("df007.csv")

df007.shape

(4118, 6)

In [17]:
hist = []
hist_file = "hist007.csv"

# first batch
train_set = torch.Tensor(train_data[:first_batch_size]).to(device)
expect_set = torch.Tensor(actual[:first_batch_size]).to(device)

print('-- train set shape: --')
print(train_set.shape)
print('-- expect set shape: --')
print(expect_set.shape)

input_size = train_set.shape[1]
model, optimizer  = prepare_model(input_size)

train(10000, train_set, expect_set, 500)

rest = len(train_data) - first_batch_size
for i in range(rest):
    clear_output(wait=True)
    print("-- loop %d / %d --" % (i, rest))

    print("-- check prediction --")
    predictions = model(train_set)
    # print("Predictions : \n", predictions.detach().cpu())
    
    error = loss_function(predictions, expect_set)
    print("Error : ", error.detach().cpu().numpy())

    next_idx = first_batch_size + i
    last_set = torch.Tensor(train_data[next_idx:next_idx + 1]).to(device)
    last_exp = actual[next_idx:next_idx + 1]
    
    print(model(last_set))
    print(last_exp)
    hist.append([model(last_set).tolist()[0][0], last_exp[0][0]])
    pd.DataFrame(hist).to_csv(hist_file, index=False)  

    train_set = torch.Tensor(train_data[i:first_batch_size + i]).to(device)
    expect_set = torch.Tensor(actual[i:first_batch_size + i]).to(device)

    start_time = time.time()
    train(800, train_set, expect_set)
    print("--- run time: %s seconds ---" % (time.time() - start_time))

-- loop 58 / 3997 --
-- check prediction --
Error :  nan
tensor([[nan]], device='mps:0', grad_fn=<LinearBackward0>)
[[0.11173606]]
0 nan
100 nan
200 nan
300 nan


KeyboardInterrupt: 

In [ ]:
L1Loss = torch.nn.L1Loss()
MSELoss = torch.nn.MSELoss()

In [ ]:
# check hist diffs
hist = pd.read_csv(hist_file)

model_result = torch.Tensor(hist.iloc[:, 0])
actual_result = torch.Tensor(hist.iloc[:, 1])

print(L1Loss(model_result, actual_result))
print(MSELoss(model_result, actual_result))

In [ ]:
l1loss_hist = []
mseloss_hist = []

for i in range(hist.shape[0]):
    ds = hist[:i]
    model_result = torch.Tensor(ds.iloc[:, 0])
    actual_result = torch.Tensor(ds.iloc[:, 1])
    l1loss_hist.append(L1Loss(model_result, actual_result))
    mseloss_hist.append(MSELoss(model_result, actual_result))

plt.plot(l1loss_hist, label="l1loss")
plt.plot(mseloss_hist, label="mseloss")
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.iloc[:, 0] - hist.iloc[:, 1])
plt.show()